In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import random_split
from torch.utils.data import Subset

In [21]:
import numpy as np
import time
import os
from PIL import Image

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [33]:

# 데이터 경로 및 하이퍼파라미터 설정
data_dir = '/kaggle/input/oasis-cnn-0825/data_for_CNN'  # 데이터 폴더 경로
batch_size = 32
num_classes = 3
num_epochs = 30
learning_rate = 0.001

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((370, 370)),  # 이미지를 충분히 크게 리사이즈
    transforms.CenterCrop(260),      # 중앙 부분을 224x224 크기로 자름
    transforms.RandomRotation(degrees=15),  # 무작위 회전 (±15도)
    transforms.RandomHorizontalFlip(),  # 무작위 수평 반전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 색상 변형
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = ImageFolder(root=data_dir, transform=transform)

# 클래스별로 Subset 생성
class_indices = [[] for _ in range(num_classes)]
for index, (image, label) in enumerate(full_dataset):
    class_indices[label].append(index)

train_indices = []
val_indices = []
for class_idx in range(num_classes):
    train_indices.extend(class_indices[class_idx][:-200])
    val_indices.extend(class_indices[class_idx][-250:])

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)

# 데이터 로더 설정
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# EfficientNet 모델 로드
model = models.efficientnet_b2(pretrained=False, num_classes=num_classes).to("cuda")



/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [34]:
learning_rate = 0.01

In [36]:
len(train_dataloader)*32, len(val_dataloader)*32

(9440, 608)

In [8]:
for images, labels in dataloader:
    # images는 배치 내 이미지들의 텐서입니다.
    # labels는 배치 내 이미지들에 해당하는 클래스 레이블입니다.
    
    # 이미지와 클래스를 확인하는 예제 코드
    for i in range(len(images)):
        image = images[i]
        label = labels[i]
        
        # 이미지와 클래스 정보 출력
        print(f"Image shape: {image.shape}, Label: {label}")

Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 0
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 0
Image shape: torch.Size([3, 224, 224]), Label: 0
Image shape: torch.Size([3, 224, 224]), Label: 0
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 1
Image shape: torch.Size([3, 224, 224]), Label: 1
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 1
Image shape: torch.Size([3, 224, 224]), Label: 0
Image shape: torch.Size([3, 224, 224]), Label: 1
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 0
Image shape: torch.Size([3, 224, 224]), Label: 2
Image shape: torch.Size([3, 224, 224]), Label: 1
Image shape: torch.S

KeyboardInterrupt: 

In [13]:
device

device(type='cuda')

각 클래스별로 1000장, 300 -> 224

Epoch [1/30], Training Loss: 1.6064978356057025
Validation Accuracy: 44.67%
Epoch [2/30], Training Loss: 0.9817075044550794
Validation Accuracy: 57.17%
Epoch [3/30], Training Loss: 0.9212817150227567
Validation Accuracy: 55.33%
Epoch [4/30], Training Loss: 0.8652829971719296
Validation Accuracy: 55.83%
Epoch [5/30], Training Loss: 0.8256980187081276
Validation Accuracy: 57.33%
Epoch [6/30], Training Loss: 0.8006651211292186
Validation Accuracy: 58.67%
Epoch [7/30], Training Loss: 0.7753435576215704
Validation Accuracy: 58.17%

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습률 스케줄러 설정
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)

num_epochs=20

best_val_loss = float('inf')  # 초기값으로 무한대 설정
best_model_dir = '/kaggle/working'  # 모델을 저장할 경로


# 훈련 및 검증 과정
for epoch in range(num_epochs):
    start_time = time.time()  # 에포크 시작 시간
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_dataloader)}')
    
    # 검증(validation) 과정
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            
            
    end_time = time.time()  # 에포크 종료 시간
    epoch_time = end_time - start_time  # 에포크 소요 시간
    
    val_loss /= len(val_dataloader)
    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')
    print(f'Validation Loss: {val_loss}, Epoch Time: {epoch_time:.2f} seconds')
    
    scheduler.step(val_loss)  # 학습률 스케줄러 업데이트
    
    
    if val_loss < best_val_loss:
        if epoch > 5:
            best_val_loss = val_loss
            best_model_path = os.path.join(best_model_dir, f'0826_10_{epoch+1}epoch.pth')
            torch.save(model.state_dict(), best_model_path)
            print(f'Saved best model with validation loss: {best_val_loss} at epoch {epoch+1}')
        
print('Training and validation finished.')

Epoch [1/20], Training Loss: 1.0951093752505416
Validation Accuracy: 52.53%
Validation Loss: 0.9407660687963167, Epoch Time: 239.66 seconds
Epoch [2/20], Training Loss: 0.9623003365629811
Validation Accuracy: 48.80%
Validation Loss: 0.9079927628238996, Epoch Time: 241.42 seconds
Epoch [3/20], Training Loss: 0.9383854106321173
Validation Accuracy: 54.13%
Validation Loss: 0.8992436404029528, Epoch Time: 240.87 seconds
Epoch [4/20], Training Loss: 0.9146840986558946
Validation Accuracy: 53.87%
Validation Loss: 0.9001684871812662, Epoch Time: 239.88 seconds
Epoch [5/20], Training Loss: 0.9112482412386749
Validation Accuracy: 58.13%
Validation Loss: 0.863901416460673, Epoch Time: 241.43 seconds
Epoch [6/20], Training Loss: 0.8980976971529298
Validation Accuracy: 55.47%
Validation Loss: 0.8844981342554092, Epoch Time: 239.88 seconds
Validation Accuracy: 55.87%
Validation Loss: 0.9120593319336573, Epoch Time: 240.99 seconds
Saved best model with validation loss: 0.9120593319336573 at epoch 7


# **Validation**

In [18]:
model = models.efficientnet_b2()
model.load_state_dict(torch.load('/kaggle/input/11epoch-oasis/best_model_epoch11.pth'))
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [11]:
def preprocess_image(image):
        val_transform = transforms.Compose([
        transforms.Resize((370, 370)),  # 이미지를 충분히 크게 리사이즈
        transforms.CenterCrop(260),      # 중앙 부분을 224x224 크기로 자름
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
        return val_transform(image).unsqueeze(0)

In [32]:
model.eval()

image_path = '/kaggle/input/surface-crack-detection/Negative/00024.jpg'
image = Image.open(image_path)


# 이미지 전처리
model.to(torch.device('cpu'))
preprocessed_image = preprocess_image(image)
preprocessed_image = preprocessed_image.to(torch.device('cpu'))


# 모델 추론
with torch.no_grad():
    outputs = model(preprocessed_image)

# 추론 결과 분석
predicted_class = torch.argmax(outputs).item()
class_names = ["class1", "class2", "class3"]  # 클래스명을 실제로 적절히 수정하세요
predicted_label = class_names[predicted_class]

print(f'Predicted class: {predicted_label}')
print(outputs)

Predicted class: class3
tensor([[-0.4456,  0.5223,  1.0447]])


In [26]:
outputs.shape, predicted_class

(torch.Size([1, 3]), 2)

In [27]:
probs = torch.nn.functional.softmax(outputs[0], dim=0)
probs

tensor([0.1624, 0.3278, 0.5098])

클래스별 2000장 + LR 스케줄러

Epoch [1/30], Training Loss: 1.2952196319052514
Validation Accuracy: 53.50%
Epoch [2/30], Training Loss: 0.9428603240150086
Validation Accuracy: 56.33%
Epoch [3/30], Training Loss: 0.8739769541202708
Validation Accuracy: 55.17%
Epoch [4/30], Training Loss: 0.8027256749411846
Validation Accuracy: 62.33%
Epoch [5/30], Training Loss: 0.7732083461386092
Validation Accuracy: 63.50%
Epoch [6/30], Training Loss: 0.7489838609670071
Validation Accuracy: 65.33%
Epoch [7/30], Training Loss: 0.7324732732265553
Validation Accuracy: 66.50%
Epoch [8/30], Training Loss: 0.7263451226531191
Validation Accuracy: 65.67%
Epoch [9/30], Training Loss: 0.7261482638881561
Validation Accuracy: 67.83%
Epoch [10/30], Training Loss: 0.7206828971492484
Validation Accuracy: 66.50%
Epoch [11/30], Training Loss: 0.7176627115683353
Validation Accuracy: 65.33%
Epoch [12/30], Training Loss: 0.7192867640802201
Validation Accuracy: 66.17%

클래스별 2500장 + 플루토

Epoch [1/30], Training Loss: 1.1889205927544453
Validation Accuracy: 54.17%
Epoch [2/30], Training Loss: 0.9099420851849495
Validation Accuracy: 62.83%
Epoch [3/30], Training Loss: 0.843723376253818
Validation Accuracy: 55.33%
Epoch [4/30], Training Loss: 0.809394041528093
Validation Accuracy: 64.33%
Epoch [5/30], Training Loss: 0.7885438896240072
Validation Accuracy: 67.00%
Epoch [6/30], Training Loss: 0.7733952780987354
Validation Accuracy: 68.67%
Epoch [7/30], Training Loss: 0.7437666062344896
Validation Accuracy: 63.67%
Epoch [8/30], Training Loss: 0.7247865476506822
Validation Accuracy: 68.67%
Epoch [9/30], Training Loss: 0.7039683355930003
Validation Accuracy: 67.83%
Epoch [10/30], Training Loss: 0.6979192223954708
Validation Accuracy: 64.33%
Epoch [11/30], Training Loss: 0.675495804751173
Validation Accuracy: 68.50%
Epoch [12/30], Training Loss: 0.6578394636194757
Validation Accuracy: 64.83%
Epoch [13/30], Training Loss: 0.6409801398185974
Validation Accuracy: 64.00%

Epoch [1/30], Training Loss: 0.9065040145890187
Validation Accuracy: 53.33%
Validation Loss: 0.967835816897844
Saved best model with validation loss: 0.967835816897844 at epoch 1
Epoch [2/30], Training Loss: 0.8331992423130294
Validation Accuracy: 61.50%
Validation Loss: 0.8213111742546684
Saved best model with validation loss: 0.8213111742546684 at epoch 2
Epoch [3/30], Training Loss: 0.8021890476598578
Validation Accuracy: 61.83%
Validation Loss: 0.79155286048588
Saved best model with validation loss: 0.79155286048588 at epoch 3
Epoch [4/30], Training Loss: 0.7564150370783725
Validation Accuracy: 65.17%
Validation Loss: 0.797015550889467
Epoch [5/30], Training Loss: 0.7408542172383454
Validation Accuracy: 63.33%
Validation Loss: 0.8201270370106948
Epoch [6/30], Training Loss: 0.731644096212872
Validation Accuracy: 67.50%
Validation Loss: 0.7349986355555685
Saved best model with validation loss: 0.7349986355555685 at epoch 6
Epoch [7/30], Training Loss: 0.7122625491376651
Validation Accuracy: 61.17%
Validation Loss: 0.8313404886346114
Epoch [8/30], Training Loss: 0.7023708483930361
Validation Accuracy: 61.50%
Validation Loss: 0.8536729185204757
Epoch [9/30], Training Loss: 0.6878132095781423
Validation Accuracy: 62.50%
Validation Loss: 0.7526584322515287
Epoch [10/30], Training Loss: 0.6767833698604067
Validation Accuracy: 62.50%
Validation Loss: 0.859652149834131
Epoch [11/30], Training Loss: 0.6659834929442001
Validation Accuracy: 65.83%
Validation Loss: 0.7337929873090041
Saved best model with validation loss: 0.7337929873090041 at epoch 11
Epoch [12/30], Training Loss: 0.6564294833247944
Validation Accuracy: 66.50%
Validation Loss: 0.737569195659537
Epoch [13/30], Training Loss: 0.6479163605277821
Validation Accuracy: 66.67%
Validation Loss: 0.7145659076540094
Saved best model with validation loss: 0.7145659076540094 at epoch 13
Epoch [14/30], Training Loss: 0.6386325454307815
Validation Accuracy: 68.33%
Validation Loss: 0.7163053766677254
Epoch [15/30], Training Loss: 0.6163983970375384
Validation Accuracy: 67.33%
Validation Loss: 0.7446301140283283
Epoch [16/30], Training Loss: 0.6113806268926394
Validation Accuracy: 65.67%
Validation Loss: 0.7817334854289105
Epoch [17/30], Training Loss: 0.5944197480961427
Validation Accuracy: 67.83%
Validation Loss: 0.7560262413401353
Epoch [18/30], Training Loss: 0.5794706932568954
Validation Accuracy: 67.33%
Validation Loss: 0.791797548532486
Epoch [19/30], Training Loss: 0.5588591801918159
Validation Accuracy: 67.50%
Validation Loss: 0.7283502283849215
Epoch [20/30], Training Loss: 0.47411285062967723
Validation Accuracy: 67.33%
Validation Loss: 0.8110448561216655
Epoch [21/30], Training Loss: 0.43115679773233706
Validation Accuracy: 68.50%
Validation Loss: 0.8360281853299392
Epoch [22/30], Training Loss: 0.41830782743833833
Validation Accuracy: 66.50%
Validation Loss: 0.8703461226664091
Epoch [23/30], Training Loss: 0.3992644193819014
Validation Accuracy: 65.00%
Validation Loss: 0.9392865096267901
Epoch [24/30], Training Loss: 0.38263551423610265
Validation Accuracy: 66.67%
Validation Loss: 0.9623668962403348
Epoch [25/30], Training Loss: 0.35678213362471534
Validation Accuracy: 61.50%
Validation Loss: 1.1999907760243667
Epoch [26/30], Training Loss: 0.3435413336097184
Validation Accuracy: 65.83%
Validation Loss: 0.9565253806741614

transforms.RandomRotation(degrees=15),  # 무작위 회전 (±15도)
    transforms.RandomHorizontalFlip(),  # 무작위 수평 반전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 색상 변형
    
    증강 추가하니까 성능 더 구려짐 ;


Epoch [1/20], Training Loss: 1.0951093752505416
Validation Accuracy: 52.53%
Validation Loss: 0.9407660687963167, Epoch Time: 239.66 seconds
Epoch [2/20], Training Loss: 0.9623003365629811
Validation Accuracy: 48.80%
Validation Loss: 0.9079927628238996, Epoch Time: 241.42 seconds
Epoch [3/20], Training Loss: 0.9383854106321173
Validation Accuracy: 54.13%
Validation Loss: 0.8992436404029528, Epoch Time: 240.87 seconds
Epoch [4/20], Training Loss: 0.9146840986558946
Validation Accuracy: 53.87%
Validation Loss: 0.9001684871812662, Epoch Time: 239.88 seconds
Epoch [5/20], Training Loss: 0.9112482412386749
Validation Accuracy: 58.13%
Validation Loss: 0.863901416460673, Epoch Time: 241.43 seconds
Epoch [6/20], Training Loss: 0.8980976971529298
Validation Accuracy: 55.47%
Validation Loss: 0.8844981342554092, Epoch Time: 239.88 seconds
Validation Accuracy: 55.87%
Validation Loss: 0.9120593319336573, Epoch Time: 240.99 seconds
Saved best model with validation loss: 0.9120593319336573 at epoch 7
Epoch [8/20], Training Loss: 0.8931057249085378
Validation Accuracy: 56.27%
Validation Loss: 0.8551276922225952, Epoch Time: 240.14 seconds
Saved best model with validation loss: 0.8551276922225952 at epoch 8
Epoch [9/20], Training Loss: 0.8883626071073241
Validation Accuracy: 55.07%
Validation Loss: 0.8619494140148163, Epoch Time: 238.74 seconds
Epoch [10/20], Training Loss: 0.8721631696668722
Validation Accuracy: 55.87%
Validation Loss: 0.8631236180663109, Epoch Time: 239.83 seconds
Epoch [11/20], Training Loss: 0.8765066476191504
Validation Accuracy: 54.13%
Validation Loss: 0.8916182344158491, Epoch Time: 238.22 seconds
Epoch [12/20], Training Loss: 0.8695619257829957
Validation Accuracy: 55.60%
Validation Loss: 0.8608725046118101, Epoch Time: 238.99 seconds
Epoch [13/20], Training Loss: 0.8638982312153962
Validation Accuracy: 56.13%
Validation Loss: 0.8819159430762132, Epoch Time: 237.90 seconds
Epoch [14/20], Training Loss: 0.8533504223419448
Validation Accuracy: 53.47%
Validation Loss: 0.9709240843852361, Epoch Time: 237.46 seconds
Epoch [15/20], Training Loss: 0.8201897629236771
Validation Accuracy: 61.20%
Validation Loss: 0.7961884985367457, Epoch Time: 237.30 seconds
Saved best model with validation loss: 0.7961884985367457 at epoch 15
Epoch [16/20], Training Loss: 0.8095878904148683
Validation Accuracy: 61.20%
Validation Loss: 0.8187295583387216, Epoch Time: 237.61 seconds
Epoch [17/20], Training Loss: 0.8028949852717124
Validation Accuracy: 59.33%
Validation Loss: 0.8198782106240591, Epoch Time: 237.27 seconds
Epoch [18/20], Training Loss: 0.8026620347621077
Validation Accuracy: 59.20%
Validation Loss: 0.8058174786468347, Epoch Time: 239.13 seconds
Epoch [19/20], Training Loss: 0.8008716684276774
Validation Accuracy: 59.07%
Validation Loss: 0.8188791858653227, Epoch Time: 238.11 seconds
Epoch [20/20], Training Loss: 0.7860904923940109
Validation Accuracy: 61.60%
Validation Loss: 0.798804797232151, Epoch Time: 238.52 seconds
Training and validation finished.